# Conformal Geometry

Conformal geometry studies *conformal structures*, or, *conformal trasnformations*.
Topology is concerned with preserving connectivity (e.g. number of holes), while
conformal geometry is concerned with preserving angles.
Note that "conformity" is a stronger notion than topology, in the sense
that a conformal equivalence implies topological equivalence.

When modelling geometric objects, we can deal with different levels of abstraction. For example,
if we want to create a type "Triangle" that encapsulates all triangles, what is the "correct" equivalence
class to use for such modelling? We know that, from the start, a triangle has three vertices with v1
connected to v2, v2 connected to v3 and v1. From an topological perspective, this view is yet lacking. We don't
know if the triangle is filled or not, hence, it could either have one hole or zero holes.

In [2]:
using Pkg, Revise
Pkg.activate("../.")

  Activating project at `~/MEGA/EMAP/Diagrams.jl`


In [31]:
using CliffordAlgebras
import CliffordAlgebras: basegrade
using LinearAlgebra: norm, normalize, dot
using Plots

function getgradesdict(cl::CliffordAlgebra)
    gradesdict = Dict(g =>[] for g in 0:sum(signature(cl)))
    ngrades = sum(signature(cl))
    for g in 0:ngrades
        for (i,p) in enumerate(propertynames(cl))
            if i <= sum(binomial.(dimcl,0:g))
                push!(gradesdict[g],p)
            end
        end
    end

    return gradesdict
end

function Base.:≈(a::MultiVector,b::MultiVector)
    vector(a) ≈ vector(b)
end

function multivector(cl::CliffordAlgebra,v::Vector)
    bases = propertynames(cl)
    mapreduce(x->x[1] * getproperty(cl,x[2]),+, zip(v,bases))
end

function Base.map(f, c::MultiVector)
    multivector(algebra(c),map(f, vector(c)))
end

function Base.reduce(op, c::MultiVector)
    reduce(op, vectorize(c))
end

"""
vectorize(c::MultiVector)

Turns a multivector into a vector of multivectors,
where each element in a multivector with a single blade.
"""
function vectorize(c::MultiVector)
    filter(x->!isapprox(norm_sqr(x),0),basevector.(Ref(algebra(c)), propertynames(c)) .* vector(c))
end

"""
unvectorize(c::MultiVector)

Turns a vector of multivectors into a multivector by
summing each component.
"""
function unvectorize(c::MultiVector)
    reduce(+,vectorize(B))
end

"""
basesfromgrade(cl::CliffordAlgebra, k::Int)

Returns a list of base multivector for a given
grade `k`. Note that grades go from ``0`` to ``n``, 
where ``n`` is the dimension, given by the sum of
the signature, e.g. ``R^{3,1,1}`` has
``n = 5``.
"""
function basesfromgrade(cl::CliffordAlgebra, k::Int)
    ngrades = sum(signature(cl))
    i = sum(binomial.(ngrades,0:k-1))+1
    j = sum(binomial.(ngrades,0:k))
    basesymbol.(Ref(cl),i:j)
end


function norm_corr(a::MultiVector)
    sqrt(abs(norm_sqr(a)))
end


bladenormalization(B::MultiVector) = B / norm_corr(B)
orthogonalproj(c::MultiVector, B::MultiVector) = (c ⨼ B) * inv(B)
orthogonalreject(a::MultiVector, B::MultiVector) = (a ∧ B) ⨽ inv(B)

"""
maxgrade(A::MultiVector)

Returns the maximum grade ``k`` of a multivector A
with non-null value, i.e. ``\\langle A \\rangle_{max}``
is equal to `grade(A,maxgrade(A))`.
"""
function maxgrade(A::MultiVector)
    D = sum(signature(algebra(A)))
    i = findlast(!isapprox(0), norm_sqr(grade(A,k)) for k in 0:D)
    if isnothing(i)
        return 0
    end
    return i - 1
end
"""
mingrade(A::MultiVector)

Returns the minimum grade ``k`` of a multivector A
with non-null value, or returns 0.
"""
function mingrade(A::MultiVector)
    D = sum(signature(algebra(A)))
    i = findfirst(!isapprox(0), norm_sqr(grade(A,k)) for k in 0:D)
    if isnothing(i)
        return 0
    end
    return i - 1
end


mingrade

In [32]:
cl = CliffordAlgebra(:CGA3D)

𝟏  = cl.𝟏
e1 = cl.e1
e2 = cl.e2
e3 = cl.e3
e₊ = cl.e₊
e₋ = cl.e₋

no = (cl.e₊ + cl.e₋)/2
n∞ = cl.e₋ - cl.e₊

propertynames(cl)

(:𝟏, :e1, :e2, :e3, :e₊, :e₋, :e1e2, :e1e3, :e2e3, :e1e₊, :e2e₊, :e3e₊, :e1e₋, :e2e₋, :e3e₋, :e₊e₋, :e1e2e3, :e1e₊e2, :e1e3e₊, :e2e₊e3, :e1e2e₋, :e1e₋e3, :e2e3e₋, :e1e₊e₋, :e2e₋e₊, :e3e₊e₋, :e1e2e3e₊, :e1e2e₋e3, :e1e2e₊e₋, :e1e3e₋e₊, :e2e3e₊e₋, :e1e2e3e₊e₋)

In [33]:
@show e₊ ⋅ e₋
@show e₊ ⋅ e₊
@show no ⋅ n∞
@show no ⋅ no
@show n∞ ⋅ n∞;

e₊ ⋅ e₋ = 0
e₊ ⋅ e₊ = +1 ∈ Cl(4, 1, 0)
no ⋅ n∞ = -1.0 ∈ Cl(4, 1, 0)
no ⋅ no = 0
n∞ ⋅ n∞ = 0


In [34]:
point(x=0,y=0,z=0) = no + x*cl.e1 + y*cl.e2 + z*cl.e3 + (x^2 + y^2 + z^2)/2 * n∞
point(;x=0,y=0,z=0) = no + x*cl.e1 + y*cl.e2 + z*cl.e3 + (x^2 + y^2 + z^2)/2 * n∞

point (generic function with 4 methods)

In [63]:
a = point()
b = point(1,1)
l1 = dual(a ∧ b ∧ n∞)

a = point(0,2)
b = point(2,2)
l2 = dual(a ∧ b ∧ n∞)

-2.0×e2e3-4.0×e3e₊-4.0×e3e₋ ∈ Cl(4, 1, 0)

In [72]:
# grade(l1*l2,5
n = sum(signature(cl))
r = mingrade(l1)
s = mingrade(l2)

# grade(l1*l2, 2n - r - s)
l1*l2

-2.0+2.0×e1e2-4.0×e1e₊+4.0×e2e₊-4.0×e1e₋+4.0×e2e₋ ∈ Cl(4, 1, 0)

In [83]:
l1

+1.0×e1e3-1.0×e2e3 ∈ Cl(4, 1, 0)

In [84]:
a

+2.0×e2-1.5×e₊+2.5×e₋ ∈ Cl(4, 1, 0)

In [85]:
n∞

-1×e₊+1×e₋ ∈ Cl(4, 1, 0)